In [397]:
import pandas as pd
import networkx as nx
import numpy as np
import json
from collections import defaultdict


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [398]:
events = pd.read_json('events\events_World_Cup.json')
matches = pd.read_json('matches\matches_World_Cup.json')
teams = pd.read_json('teams.json')

In [399]:
def load_public_dataset(tournament='World_Cup'):
    """
    Load the json files with the matches, events, players and competitions
    
    Parameters
    ----------
    data_folder : str, optional
        the path to the folder where json files are stoyellow.
        
    tournaments : list, optional
        the list of tournaments to load. 
        
    Returns
    -------
    tuple
        a tuple of four dictionaries, containing matches, events, players and competitions
        
    """
    # loading the matches and events data
    matches, events = {}, {}
    with open('events/events_%s.json' %tournament) as json_data:
        events = json.load(json_data)
    with open('matches/matches_%s.json' %tournament) as json_data:
        matches = json.load(json_data)
    
    match_id2events = defaultdict(list)
    match_id2match = defaultdict(dict)
    for event in events:
        match_id = event['matchId']
        match_id2events[match_id].append(event)
                                         
    for match in matches:
        match_id = match['wyId']
        match_id2match[match_id] = match

    # loading the players data
    with open('players.json') as json_data:
        players = json.load(json_data)
    
    player_id2player = defaultdict(dict)
    for player in players:
        player_id = player['wyId']
        player_id2player[player_id] = player
    
    # loading the competitions data
    teams={}
    with open('teams.json') as json_data:
        teams = json.load(json_data)
    team_id2team = defaultdict(dict)
    for team in teams:
        team_id = team['wyId']
        team_id2team[team_id] = team
    
    return match_id2match, match_id2events, player_id2player, team_id2team

In [400]:
match_id2match, match_id2events, player_id2player, team_id2team = load_public_dataset()

In [401]:
events.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],122671,"[{'y': 50, 'x': 50}, {'y': 53, 'x': 35}]",2057954,Pass,16521,1H,1.656214,85,258612104
1,8,High pass,[{'id': 1801}],139393,"[{'y': 53, 'x': 35}, {'y': 19, 'x': 75}]",2057954,Pass,16521,1H,4.487814,83,258612106
2,1,Air duel,"[{'id': 703}, {'id': 1801}]",103668,"[{'y': 81, 'x': 25}, {'y': 83, 'x': 37}]",2057954,Duel,14358,1H,5.937411,10,258612077
3,1,Air duel,"[{'id': 701}, {'id': 1802}]",122940,"[{'y': 19, 'x': 75}, {'y': 17, 'x': 63}]",2057954,Duel,16521,1H,6.406961,10,258612112
4,8,Simple pass,[{'id': 1801}],122847,"[{'y': 17, 'x': 63}, {'y': 15, 'x': 71}]",2057954,Pass,16521,1H,8.562167,85,258612110


In [402]:
#fazer o mesmo do df das yellowes para esse aqui (criar df com features)

df = matches[['wyId']]
df = df.rename(columns={'wyId':'matchID'})

#colocar team1 e team2
home_l = []
away_l = []

for match in matches['teamsData']:
    t0 = list(match.values())[0]
    t1 = list(match.values())[1]

    if t0['side'] == 'home':
        home_l.append(t0['teamId'])
        away_l.append(t1['teamId'])  
    else: 
        away_l.append(t0['teamId'])
        home_l.append(t1['teamId'])

df['team1_ID'] = home_l
df['team2_ID'] = away_l

df.head()

,matchID,team1_ID,team2_ID
0,2058017,4418,9598
1,2058016,5629,2413
2,2058015,9598,2413
3,2058014,4418,5629
4,2058012,14358,9598


In [403]:
df.shape

(64, 3)

In [404]:
# adaptando coluna 'tags' para conseguir interpretar se evento foi bem sucedido ou não. 'id' = 1801 ou 1802
def converting_tags(aux):
    new = pd.Series()
    try:
        tags = []
        for tag in aux:
            tags.append(list(tag.values())[0])
        new = tags
    except IndexError:
        return None

    return new

events['tags'] = events['tags'].map(converting_tags)
#events['ev_success'] = events['ev_success'].replace([1801, 1802], [1, 0])

C:\Users\eduar\AppData\Local\Temp/ipykernel_26556/4023803042.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new = pd.Series()


In [405]:
# GK_SAVES

#contar quantos subEvents = 'Save attempt' com tag = 1801(bem sucedido) para todos os (match, team)

gk_saves = events[(events['subEventName'] == 'Save attempt') & (1801 in events['tags'])].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
gk_saves.columns = gk_saves.columns.droplevel(0)
gk_saves = gk_saves.reset_index().rename(columns={'count':'gk_saves'})
gk_saves.head()

#juntar feature com df
df = df.merge(gk_saves, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(gk_saves, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"gk_saves_x": "gk_saves_T1", "gk_saves_y": "gk_saves_T2"})

df = df.fillna(0)
df

,matchID,team1_ID,team2_ID,gk_saves_T1,gk_saves_T2
0,2058017,4418,9598,0.0,4.0
1,2058016,5629,2413,5.0,0.0
2,2058015,9598,2413,1.0,3.0
3,2058014,4418,5629,1.0,1.0
4,2058012,14358,9598,1.0,6.0
5,2058013,7047,2413,0.0,0.0
6,2058011,6380,5629,1.0,5.0
7,2058010,15670,4418,1.0,1.0
8,2058009,12430,2413,1.0,3.0
9,2058008,7047,6697,3.0,0.0


In [425]:
events['eventName'] == 'Foul'

False

In [427]:
#RED_CARDS

#contar quantos subEvents = 'Foul' com tag = 1701(red card) ou 1703(second yellow card) para todos os (match, team)

red_card = events[(events['eventName'] == 'Foul') & ((1701 in events['tags']) | (1703 in events['tags']))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
# red_card = events.query('(eventName == "Foul") & ((1701, 1703) in tags)').groupby(['matchId', 'teamId']).agg({'eventId':['count']})
# red_card.columns = red_card.columns.droplevel(0)
# red_card = red_card.reset_index().rename(columns={'count':'red_card'})
red_card
# #juntar feature com df
# df = df.merge(red_card, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
# df = df.merge(red_card, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

# df = df.rename(columns={"red_card_x": "red_card_T1", "red_card_y": "red_card_T2"})

# df = df.fillna(0)
# df


eventId
                 count
matchId teamId        
2057954 14358       22
        16521       10
2057955 15670        6
        16129       12
2057956 14358       10
        16129       10
2057957 15670       10
        16521       13
2057958 14358       19
        15670       17
2057959 16129       16
        16521        7
2057960 1598        10
        9905        12
2057961 10840       14
        16216       22
2057962 9905        19
        16216       22
2057963 1598        14
        10840       14
2057964 9905        12
        10840       18
2057965 1598         5
        16216       20
2057966 4418        16
        8493        19
2057967 7712        18
        15594       10
2057968 4418        11
        15594       15
2057969 7712         8
        8493         5
2057970 4418        10
        7712        10
2057971 8493        12
        15594       12
2057972 7839        15
        12274       10
2057973 9598        20
        16823       16
2057974 9598        22
        12274       17
2057975 7839        10
        16823        9
2057976 12274       16
        16823       20
2057977 7839        11
        9598        11
2057978 6380        11
        6697        19
2057979 16871       17
        17322       14
2057980 6380        13
        16871       12
2057981 6697        13
        17322       18
2057982 6380         7
        17322       14
2057983 6697         9
        16871       16
2057984 3148        10
        15473       18
2057985 7047        20
        14855       23
2057986 3148        12
        7047        14
2057987 14855       23
        15473        7
2057988 3148         7
        14855       16
2057989 7047        11
        15473       14
2057990 5629        17
        17929       20
2057991 2413         8
        16276       14
2057992 5629        12
        16276       13
2057993 2413        13
        17929       14
2057994 2413        11
        5629        14
2057995 16276       19
        17929       18
2057996 13869        6
        19314       15
2057997 12430       15
        12913       10
2057998 12430        9
        13869       15
2057999 12913        8
        19314       15
2058000 12913       11
        13869        8
2058001 12430       15
        19314       15
2058002 9905        14
        15670       13
2058003 4418        23
        12274       16
2058004 1598         5
        14358       20
2058005 7712        19
        9598         5
2058006 6380         7
        15473       18
2058007 5629        13
        12913        9
2058008 6697        13
        7047        11
2058009 2413        14
        12430       25
2058010 4418        16
        15670       18
2058011 5629        16
        6380        14
2058012 9598        19
        14358       24
2058013 2413         7
        7047        11
2058014 4418         8
        5629        16
2058015 2413        15
        9598        24
2058016 2413         5
        5629        12
2058017 4418        13
        9598        13

In [414]:
df['red_card_T2'].value_counts()

0.0    64
Name: red_card_T2, dtype: int64

In [ ]:
# **** arrumar esta merda ******** (acho que se colocar .query funciona)

#YELLOW_CARDS

#contar quantos subEvents = 'Foul' com tag = 1702(yellow card) ou 1703(second yellow card) para todos os (match, team)

yellow_card = events.query('(eventName == "Foul") & ((1702, 1703) in last_tag)').groupby(['matchId', 'teamId']).agg({'eventId':['count']})
yellow_card.columns = yellow_card.columns.droplevel(0)
yellow_card = yellow_card.reset_index().rename(columns={'count':'yellow_card'})

#juntar feature com df
df = df.merge(yellow_card, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(yellow_card, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"yellow_card_x": "yellow_card_T1", "yellow_card_y": "yellow_card_T2"})

df = df.fillna(0)
df

,matchID,team1_ID,team2_ID,gk_saves_T1,gk_saves_T2,red_card_T1,red_card_T2,yellow_card_T1,yellow_card_T2
0,2058017,4418,9598,0.0,2.0,0.0,0.0,2.0,1.0
1,2058016,5629,2413,5.0,0.0,0.0,0.0,0.0,1.0
2,2058015,9598,2413,0.0,3.0,0.0,0.0,1.0,0.0
3,2058014,4418,5629,1.0,1.0,0.0,0.0,0.0,3.0
4,2058012,14358,9598,1.0,5.0,0.0,0.0,1.0,2.0
5,2058013,7047,2413,0.0,0.0,0.0,0.0,1.0,1.0
6,2058011,6380,5629,1.0,5.0,0.0,0.0,2.0,2.0
7,2058010,15670,4418,0.0,1.0,0.0,0.0,1.0,1.0
8,2058009,12430,2413,1.0,3.0,0.0,0.0,3.0,1.0
9,2058008,7047,6697,3.0,0.0,0.0,1.0,1.0,2.0


In [ ]:
#ASSISTS

In [ ]:
events['eventName'].value_counts()

Pass                       56457
Duel                       25927
Others on the ball          9282
Free Kick                   5964
Foul                        1766
Shot                        1419
Save attempt                 560
Goalkeeper leaving line      212
Offside                      172
Name: eventName, dtype: int64

In [428]:
events[events['eventName'] == 'Foul']['tags'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]        1537
[1702]     220
[1901]       5
[1703]       2
[1701]       2
Name: tags, dtype: int64